# 04 - Création d'un dataset des données nettoyées de job_events (après fractionnement)

Ce notebook génère 1 csv :

- clean_merge_job_events_dataset.csv

Etapes :

- Suppression des lignes avec les valeurs timestamp_start manquantes

- Suppression des colonnes ayant des valeurs uniques

- Remplacement des données maquantes

- Conversion des types des séries en fonction des types des valeurs

# A. Imports

## a) Librairies

In [1]:
import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## b) Données

In [2]:
# source path to raw metrics dataset
filename = 'raw_merge_job_events_dataset.csv'
path = '../data/jobs'
job_events = os.path.join(path, filename)
# target path to save merge raw metrics dataset
save_csv = '../data/jobs/clean_merge_job_events_dataset.csv'

# B. Dataframe

## a) Création

In [3]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(job_events, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 1139
Data columns (total 32 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   jobId                                                 1140 non-null   int64  
 1   timestamp_start                                       1139 non-null   object 
 2   jsonVersion                                           1139 non-null   float64
 3   totalCopies_start                                     1139 non-null   float64
 4   LED_iper                                              1139 non-null   float64
 5   bars_iper                                             1139 non-null   object 
 6   drops_iper                                            1139 non-null   float64
 7   dithering_iper                                        1139 non-null   object 
 8   deadPixelsOffset_iper                                 1139

In [4]:
# réindexation
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df.head(5)

,jobId,timestamp_start,jsonVersion,totalCopies_start,LED_iper,bars_iper,drops_iper,dithering_iper,deadPixelsOffset_iper,level_user,...,power_irDryers,power_uvDryers,leftMargin_registration_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,timestamp_end,totalCopies_end,consumption_operatorSideTanks_varnishConsumption
0,1704202867,2024-01-02T13:41:07.4132382Z,2.0,350.0,10.0,"[1, 2]",1.0,False,0.0,Operator,...,45.0,100.0,0.0,16.0,16.0,1.0,SUCCESS,2024-01-02T13:42:21.6136182Z,0.0,0.0
1,1704204476,2024-01-02T14:07:56.4227111Z,2.0,350.0,10.0,"[1, 2]",1.0,False,0.0,Operator,...,45.0,100.0,0.0,16.0,16.0,1.0,SUCCESS,2024-01-02T14:13:05.3979233Z,0.0,0.0
2,1704205571,2024-01-02T14:26:11.4978019Z,2.0,350.0,10.0,"[1, 2]",1.0,False,0.0,Operator,...,45.0,100.0,0.0,16.0,16.0,1.0,CANCELED,2024-01-02T14:29:28.9464837Z,0.0,0.0
3,1704206174,2024-01-02T14:36:14.5325077Z,2.0,350.0,10.0,"[1, 2]",1.0,False,0.0,Operator,...,45.0,100.0,0.0,16.0,16.0,1.0,CANCELED,2024-01-02T14:38:22.8471710Z,0.0,0.0
4,1704211006,2024-01-02T15:56:46.3395936Z,2.0,350.0,10.0,"[1, 2]",1.0,False,0.0,Operator,...,45.0,100.0,0.0,16.0,16.0,1.0,SUCCESS,2024-01-02T16:01:48.2788709Z,0.0,0.0


## b) Nettoyage

### 1. Suppression de lignes

In [5]:
# on cherche les index des lignes sans timestamp de départ
no_start_row = list(df[df['timestamp_start'].isna()].index)
no_start_row

[1139]

In [6]:
# on supprime les lignes sans timestamp de départ
df.drop(no_start_row, axis=0, inplace=True)

### 2. Suppression de colonnes

In [7]:
# liste des colonnes
df.columns

Index(['jobId', 'timestamp_start', 'jsonVersion', 'totalCopies_start',
       'LED_iper', 'bars_iper', 'drops_iper', 'dithering_iper',
       'deadPixelsOffset_iper', 'level_user', 'operator_user', 'speed_ifoil',
       'enabled_ifoil', 'optifoil_ifoil', 'stampAreas_ifoil',
       'heater1Enabled_ifoil', 'heater1Temperature_ifoil',
       'x_imageLayout_layout', 'y_imageLayout_layout',
       'width_paperFormat_layout', 'height_paperFormat_layout', 'speed_layout',
       'power_irDryers', 'power_uvDryers',
       'leftMargin_registration_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'mode_remoteScannerRegistration', 'jobState', 'timestamp_end',
       'totalCopies_end', 'consumption_operatorSideTanks_varnishConsumption'],
      dtype='object')

In [8]:
# liste des types des colonnes
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [9]:
# suppression des colonnes non pertinentes
# col_to_drop = [
#     'path', 
#     'image']
# df.drop(col_to_drop, axis=1, inplace=True)

KeyError: "['path', 'image'] not found in axis"

### 3. Données maquantes

In [10]:
# on liste les colonnes qui contiennent des valeurs nulles
col_with_nan = []
for col in df.columns:
    if df[col].isnull().any():
        col_with_nan.append(col)
        print(f'{col} : {df[col].unique()}') if df[col].nunique() <= 10 else print(f'{col} : {df[col].nunique()} - dtype : {df[col].dtype}')     

jobState : ['SUCCESS' 'CANCELED' 'ERROR' nan]
timestamp_end : 1131 - dtype : object
totalCopies_end : 243 - dtype : float64
consumption_operatorSideTanks_varnishConsumption : 984 - dtype : float64


In [12]:
# on remplace les 'jobState' non défini par UNDEFINED
df['jobState'] = df['jobState'].fillna('UNDEFINED')

In [13]:
# on converti les valeur de 'timestamp_end' en booléen
#df['timestamp_end'] = df['timestamp_end'].apply(lambda x: False if pd.isna(x) else True)
df['timestamp_end'] = df['timestamp_end'].fillna(df['timestamp_start'])

In [14]:
# on remplace les valeurs de 'totalCopies_end' nulles par 0
df['totalCopies_end'] = df['totalCopies_end'].fillna(float(0))

In [15]:
# on remplace les valeurs de 'varnishConsumption' nulles par 0
df['consumption_operatorSideTanks_varnishConsumption'] = df['consumption_operatorSideTanks_varnishConsumption'].fillna(float(0))

### 4. Conversion de type

In [16]:
# conversion des colonnes de type 'float' dont toutes les valeurs ont des décimales à 0 en type 'int'
for col in list(df.select_dtypes(exclude=['object','bool']).columns):
    if not True in (math.modf(value)[0] != float(0) for value in list(df[col].unique())):
        df[col] = df[col].astype('int64')

In [17]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in list(df.select_dtypes(include=['object']).columns):
    print(f"{col} : {df[col].unique()}") if df[col].nunique() <= 5  else print(f"{col} : {df[col].nunique()} values")

timestamp_start : 1139 values
bars_iper : ['[1, 2]' '[2]' '[1]']
dithering_iper : [False True]
level_user : ['Operator' 'Distributor']
operator_user : ['JAN' 'Micka' 'Viktor' 'Distributor']
enabled_ifoil : [False True]
optifoil_ifoil : [True False]
stampAreas_ifoil : 9 values
heater1Enabled_ifoil : [False True]
jobState : ['SUCCESS' 'CANCELED' 'ERROR' 'UNDEFINED']
timestamp_end : 1139 values


In [18]:
# on peut convertir certaines colonnes au format datetime
df['timestamp_start'] = pd.to_datetime(df['timestamp_start'], utc=True)
df['timestamp_end'] = pd.to_datetime(df['timestamp_end'], utc=True)

In [19]:
# on peut convertir le type des colonnes contenant True au format booléen
for col in list(df.select_dtypes(include=['object']).columns):
    values = list(df[col].unique())
    if True in (len(values) == 2  and value == True for value in values):
        df[col] = df[col].astype('bool')

In [20]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in list(df.select_dtypes(include=['object']).columns):
    print(f"{col} : {df[col].unique()}") if df[col].nunique() <= 5  else print(f"{col} : {df[col].nunique()} values")

bars_iper : ['[1, 2]' '[2]' '[1]']
level_user : ['Operator' 'Distributor']
operator_user : ['JAN' 'Micka' 'Viktor' 'Distributor']
stampAreas_ifoil : 9 values
jobState : ['SUCCESS' 'CANCELED' 'ERROR' 'UNDEFINED']


### 5. Output csv

In [21]:
df.columns

Index(['jobId', 'timestamp_start', 'jsonVersion', 'totalCopies_start',
       'LED_iper', 'bars_iper', 'drops_iper', 'dithering_iper',
       'deadPixelsOffset_iper', 'level_user', 'operator_user', 'speed_ifoil',
       'enabled_ifoil', 'optifoil_ifoil', 'stampAreas_ifoil',
       'heater1Enabled_ifoil', 'heater1Temperature_ifoil',
       'x_imageLayout_layout', 'y_imageLayout_layout',
       'width_paperFormat_layout', 'height_paperFormat_layout', 'speed_layout',
       'power_irDryers', 'power_uvDryers',
       'leftMargin_registration_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'mode_remoteScannerRegistration', 'jobState', 'timestamp_end',
       'totalCopies_end', 'consumption_operatorSideTanks_varnishConsumption'],
      dtype='object')

In [22]:
df.to_csv(path_or_buf=Path(save_csv))